In [5]:
!pip install --quiet flwr

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
from skimage.feature import hog
import multiprocessing
import torch
import socket
import pickle
from collections import defaultdict
import threading
import time
from multiprocessing import Process, Queue
# Function to aggregate weights
import flwr as fl

In [6]:
# Hyperparameters
input_dim = 324
output_dim = 10
learning_rate = 0.001
epochs = 1000

# Download MNIST dataset

In [7]:
# 1. Load and Preprocess CIFAR10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization to [-1, 1]
])

full_train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# 2. Create Train-Validation Split (80-20 Split)
train_size = int(0.8 * len(full_train_data))
val_size = len(full_train_data) - train_size
train_data, val_data = random_split(full_train_data, [train_size, val_size])
# 2. Flatten Images
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
val_loader = DataLoader(val_data, batch_size=100, shuffle=False)
test_loader = DataLoader(test_data, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
full_train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

# Preprocess MNIST DATASET

In [9]:
# 3. Preprocess and Extract HOG Features
def extract_hog_features(loader):
    hog_features = []
    labels = []
    
    for img_batch, label_batch in loader:
        # Flatten each image to (batch_size, 3, 1024) and average RGB channels to get shape (batch_size, 1024)
        img_batch = img_batch.view(img_batch.size(0), 3, 1024).mean(dim=1)  # Average RGB channels
        
        # Reshape to (32, 32) and extract HOG features
        for img, label in zip(img_batch, label_batch):
            img_reshaped = img.view(32, 32).numpy()  # Reshape to 32x32
            
            # HOG feature extraction
            hog_feat = hog(img_reshaped, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
            hog_features.append(hog_feat)
            labels.append(label)
    
    return torch.tensor(hog_features), torch.tensor(labels)

train_images, train_labels = extract_hog_features(train_loader)
val_images, val_labels = extract_hog_features(val_loader)
test_images, test_labels = extract_hog_features(test_loader)

/tmp/ipykernel_506/3617597787.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(hog_features), torch.tensor(labels)


# Model Architecture and Training Config

In [10]:
# Linear Regression Model using PyTorch
class SoftmaxRegression(nn.Module):
    def __init__(self, input_dim , output_dim):
        super(SoftmaxRegression, self).__init__()
        # Define a linear layer (input_dim -> 1)
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, X):
        # Forward pass: apply the linear layer
        return self.linear(X)

In [11]:
# Initialize the model, loss function, and optimizer
model = SoftmaxRegression(input_dim , output_dim)
criterion = nn.CrossEntropyLoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and Evaluate Model

In [12]:
# 5. Train the Model
def train(model, data, labels, criterion, optimizer):
        model.train()
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
#         print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        # Evaluate on training data every epoch
        accuracy, f1 = evaluate(model, data, labels)
        return accuracy , f1 , model.state_dict()

# 6. Evaluation Function (Accuracy and F1-score)
def evaluate(model, data, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.numpy()
        labels = labels.numpy()
        
    # Calculate accuracy and F1-score
    accuracy = accuracy_score(labels, predicted) * 100
    f1 = f1_score(labels, predicted, average='weighted')
    return accuracy, f1

# 7. Test Function to Assess Model on Test Set
def test(model, test_data, test_labels):
    accuracy, f1 = evaluate(model, test_data, test_labels)
    print(f'Test Accuracy: {accuracy:.2f}%, Test F1-score: {f1:.2f}')
        
test(model , test_images , test_labels)

Test Accuracy: 7.62%, Test F1-score: 0.04


In [14]:

# Initialize a variable to keep track of the best validation accuracy
best_val_acc = 0.0  # Set to 0 initially
best_model_path = 'best_model.pth'  # File path to save the best model

for epoch in range(epochs):
    # Train the model for one epoch
    train_acc, train_f1 , _ = train(model, train_images, train_labels, criterion, optimizer)
    
    # Evaluate on the validation set
    val_acc, val_f1  = evaluate(model, val_images, val_labels)
    
    # Print training and validation metrics for the current epoch
    print(f'Epoch [{epoch+1}/{epochs}] - Train Accuracy: {train_acc:.2f}%, Train F1-score: {train_f1:.2f}')
    print(f'Epoch [{epoch+1}/{epochs}] - Val Accuracy: {val_acc:.2f}%, Val F1-score: {val_f1:.2f}')
    
    # Check if the current validation accuracy is the best so far
    if val_acc > best_val_acc:
        best_val_acc = val_acc  # Update the best validation accuracy
        torch.save(model.state_dict(), best_model_path)  # Save the model's state_dict
        print(f'New best model saved with Val Accuracy: {best_val_acc:.2f}%')

# Load the best model after training is completed
model.load_state_dict(torch.load(best_model_path))

# Test the best model on the test set
# test(model, X_test_tensor, y_test_tensor)


Epoch [1/1000] - Train Accuracy: 10.91%, Train F1-score: 0.06
Epoch [1/1000] - Val Accuracy: 10.60%, Val F1-score: 0.06
New best model saved with Val Accuracy: 10.60%
Epoch [2/1000] - Train Accuracy: 12.29%, Train F1-score: 0.08
Epoch [2/1000] - Val Accuracy: 12.08%, Val F1-score: 0.08
New best model saved with Val Accuracy: 12.08%
Epoch [3/1000] - Train Accuracy: 13.74%, Train F1-score: 0.09
Epoch [3/1000] - Val Accuracy: 13.59%, Val F1-score: 0.10
New best model saved with Val Accuracy: 13.59%
Epoch [4/1000] - Train Accuracy: 15.23%, Train F1-score: 0.11
Epoch [4/1000] - Val Accuracy: 15.19%, Val F1-score: 0.11
New best model saved with Val Accuracy: 15.19%
Epoch [5/1000] - Train Accuracy: 16.71%, Train F1-score: 0.13
Epoch [5/1000] - Val Accuracy: 16.68%, Val F1-score: 0.13
New best model saved with Val Accuracy: 16.68%
Epoch [6/1000] - Train Accuracy: 18.18%, Train F1-score: 0.14
Epoch [6/1000] - Val Accuracy: 17.80%, Val F1-score: 0.14
New best model saved with Val Accuracy: 17.80

/tmp/ipykernel_455/2234702466.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


<All keys matched successfully>

In [15]:
test(model, test_images, test_labels)

Test Accuracy: 47.93%, Test F1-score: 0.48


# Federate Training

## Devide data to train locally

In [13]:
# Class mappings for CIFAR-10
from torch.utils.data import DataLoader, Subset

class_map = {
    0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat',
    4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'
}
# Specify the class indices you want for the filtered dataset
selected_classes = [1, 2, 3, 4, 5, 6, 7, 8, 9]  # ô tô, chim, mèo, hươu, chó, ếch, ngựa, tàu, xe tải
# Filter dataset by selected classes
def filter_dataset_by_class(dataset, class_indices):
    filtered_indices = [i for i, (_, label) in enumerate(dataset) if label in class_indices]
    return Subset(dataset, filtered_indices)

### client 1 data

In [14]:
#filter dataset
client1_train_data = filter_dataset_by_class(full_train_data, selected_classes)
# split the train and val
client1_train_size = int(0.8 * len(client1_train_data))
client1_val_size = len(client1_train_data) - client1_train_size
client1_train_data, client1_val_data = random_split(client1_train_data, [client1_train_size, client1_val_size])
# move to dataloader
client1_train_dataloader = DataLoader(client1_train_data, batch_size=100, shuffle=True)
client1_val_dataloader = DataLoader(client1_val_data, batch_size=100, shuffle=True)


client1_train_images, client1_train_labels = extract_hog_features(client1_train_dataloader)
client1_val_images, client1_val_labels = extract_hog_features(client1_val_dataloader)

# Optionally, print out a summary for verification
print(f"Client 2 Train Dataset: {len(client1_train_data)} samples")
print(f"Client 2 Val Dataset: {len(client1_val_data)} samples")

Client 2 Train Dataset: 36000 samples
Client 2 Val Dataset: 9000 samples


### client 2 data

In [15]:
# Classes selected for client 2 (all classes)
client2_selected_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Filter dataset for client 2
client2_train_data = filter_dataset_by_class(full_train_data, client2_selected_classes)

# Split the train and val datasets for client 2
client2_train_size = int(0.8 * len(client2_train_data))
client2_val_size = len(client2_train_data) - client2_train_size
client2_train_data, client2_val_data = random_split(client2_train_data, [client2_train_size, client2_val_size])

# Move client 2 datasets to DataLoader
client2_train_dataloader = DataLoader(client2_train_data, batch_size=100, shuffle=True)
client2_val_dataloader = DataLoader(client2_val_data, batch_size=100, shuffle=True)

# Extract HOG features for client 2's training and validation data
client2_train_images, client2_train_labels = extract_hog_features(client2_train_dataloader)
client2_val_images, client2_val_labels = extract_hog_features(client2_val_dataloader)

# Optionally, print out a summary for verification
print(f"Client 2 Train Dataset: {len(client2_train_data)} samples")
print(f"Client 2 Val Dataset: {len(client2_val_data)} samples")


Client 2 Train Dataset: 40000 samples
Client 2 Val Dataset: 10000 samples


## Federate Set up

In [23]:
LOCAL_HOST_ADDRESS = 3015
DATASIZE = 15000
ROUNDS = 10

In [24]:
# Function to aggregate weights
import concurrent.futures

def aggregate_weights(client_weights):
    aggregated_weights = defaultdict(float)
    num_clients = len(client_weights)
    for weights in client_weights:
        for key, value in weights.items():
            aggregated_weights[key] += value / num_clients
    return dict(aggregated_weights)

        
# Function to handle individual client connections
def handle_client(client_socket, client_weights, lock):
    try:
        data = client_socket.recv(DATASIZE)
        with lock:
            client_weights.append(pickle.loads(data))
    #client_socket.close()
    except socket.timeout:
        print(f"Timeout from Client {client_socket}")
    except Exception as e:
        print(f"Error handling client {client_socket}: {e}")
    
# Server function to handle federated training
def server_process(num_clients, rounds=ROUNDS):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', LOCAL_HOST_ADDRESS))
    server_socket.listen(num_clients)
    lock = threading.Lock()
    for round in range(rounds):
        client_weights = []        
        # Use ThreadPoolExecutor to handle multiple clients concurrently
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_clients) as executor:
            futures = []
            for _ in range(num_clients):
                client_socket, _ = server_socket.accept()
                futures.append(executor.submit(handle_client, client_socket, client_weights, lock))
            # Wait for all clients to send their data
            concurrent.futures.wait(futures)
        # Aggregate weights
        aggregated_weights = aggregate_weights(client_weights)
        # Send aggregated weights back to each clientt
        for _ in range(num_clients):
            client_socket, addr = server_socket.accept()
            with client_socket:
                client_socket.sendall(pickle.dumps(aggregated_weights))
                
        print(f"Server: Round {round+1} complete. Model sent to clients.")
    
    server_socket.close()


In [25]:
# Client function for local training and communication with server
criterion = nn.CrossEntropyLoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model = SoftmaxRegression(input_dim , output_dim)
def client_process(client_id, data, labels,val_images, val_labels , model , criterion, optimizer, rounds=ROUNDS):
    server_address = ('localhost', LOCAL_HOST_ADDRESS)
    for round in range(rounds):
        # Train locally
        local_acc , local_f1 , local_weights = train(model, data, labels, criterion, optimizer)
        print (f"Client {client_id} trained - Accuracy {local_acc} - F1 {local_f1}")
        local_val_acc , local_val_f1 = evaluate(model, val_images, val_labels)
        print (f"Client {client_id}  -  Validation Accuracy {local_acc} -Validation F1 {local_f1}")
        # Connect to server and send weights
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.connect(server_address)
            sock.sendall(pickle.dumps(local_weights))
            ack = sock.recv(1024)  # Wait for acknowledgment from server
        # Receive the aggregated model from server
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.connect(server_address)
            aggregated_weights = pickle.loads(sock.recv(DATASIZE))
            model.load_state_dict(aggregated_weights)
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Reinitialize optimizer

In [26]:
# Creating threads for server and clients
def run_federated_training():
    # Assume data_loaders are predefined data loaders for each client
#     data_loader_1 = ...
#     data_loader_2 = ...
#     data_loader_3 = ...

    # Start server thread
    server_thread = threading.Thread(target=server_process, args=(2,))
    server_thread.start()
    time.sleep(1)  # Wait a moment to ensure server starts before clients connect

    # Start client threads
    client_threads = [
        threading.Thread(target=client_process, args=(1, client1_train_images , client1_train_labels ,client1_val_images, client1_val_labels , model, criterion , optimizer)),
        threading.Thread(target=client_process, args=(2, client2_train_images , client2_train_labels ,client2_val_images, client2_val_labels, model , criterion , optimizer)),
        #threading.Thread(target=client_process, args=(3, train_images , train_labels, criterion , optimizer)),
    ]
    for thread in client_threads:
        thread.start()
    # Join threads to ensure complete execution
    for thread in client_threads:
        thread.join()
    server_thread.join()

In [27]:
# Run the federated training process
run_federated_training()

Client 1 trained - Accuracy 10.555555555555555 - F1 0.05106715293152026
Client 2 trained - Accuracy 9.432500000000001 - F1 0.04245904302048501
Client 1  -  Validation Accuracy 10.555555555555555 -Validation F1 0.05106715293152026
Client 2  -  Validation Accuracy 9.432500000000001 -Validation F1 0.04245904302048501
Server: Round 1 complete. Model sent to clients.
Client 1 trained - Accuracy 13.675 - F1 0.07597420659147312
Client 2 trained - Accuracy 13.0 - F1 0.07054366376063935
Client 1  -  Validation Accuracy 13.675 -Validation F1 0.07597420659147312
Client 2  -  Validation Accuracy 13.0 -Validation F1 0.07054366376063935
Server: Round 2 complete. Model sent to clients.
Client 1 trained - Accuracy 17.544444444444444 - F1 0.13759623361192105
Client 2 trained - Accuracy 15.937499999999998 - F1 0.11829502942469854
Client 1  -  Validation Accuracy 17.544444444444444 -Validation F1 0.13759623361192105
Client 2  -  Validation Accuracy 15.937499999999998 -Validation F1 0.11829502942469854
Se

/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/opt/conda/lib/python3.10/site-p

Client 1  -  Validation Accuracy 18.641666666666666 -Validation F1 0.15691202060936635Client 2 trained - Accuracy 17.34 - F1 0.13970660700294624

Client 2  -  Validation Accuracy 17.34 -Validation F1 0.13970660700294624
Server: Round 4 complete. Model sent to clients.
Client 1 trained - Accuracy 20.366666666666667 - F1 0.18070743531907185
Client 1  -  Validation Accuracy 20.366666666666667 -Validation F1 0.18070743531907185
Client 2 trained - Accuracy 19.0475 - F1 0.16135237691322335
Client 2  -  Validation Accuracy 19.0475 -Validation F1 0.16135237691322335
Server: Round 5 complete. Model sent to clients.
Client 1 trained - Accuracy 23.080555555555556 - F1 0.21571840144859425
Client 1  -  Validation Accuracy 23.080555555555556 -Validation F1 0.21571840144859425
Client 2 trained - Accuracy 21.087500000000002 - F1 0.18566614625036704
Client 2  -  Validation Accuracy 21.087500000000002 -Validation F1 0.18566614625036704
Server: Round 6 complete. Model sent to clients.
Client 2 trained - 

/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/opt/conda/lib/python3.10/site-p

Client 2 trained - Accuracy 24.7575 - F1 0.22549994974926757
Client 1 trained - Accuracy 27.02222222222222 - F1 0.2616584019303256
Client 2  -  Validation Accuracy 24.7575 -Validation F1 0.22549994974926757
Client 1  -  Validation Accuracy 27.02222222222222 -Validation F1 0.2616584019303256
Server: Round 8 complete. Model sent to clients.
Client 1 trained - Accuracy 28.530555555555555 - F1 0.27835435602029135
Client 2 trained - Accuracy 26.279999999999998 - F1 0.2411515926602877
Client 1  -  Validation Accuracy 28.530555555555555 -Validation F1 0.27835435602029135
Client 2  -  Validation Accuracy 26.279999999999998 -Validation F1 0.2411515926602877
Server: Round 9 complete. Model sent to clients.
Client 1 trained - Accuracy 30.172222222222224 - F1 0.29562047702337907
Client 2 trained - Accuracy 27.525 - F1 0.25370406995286543
Client 1  -  Validation Accuracy 30.172222222222224 -Validation F1 0.29562047702337907
Client 2  -  Validation Accuracy 27.525 -Validation F1 0.25370406995286543


/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/opt/conda/lib/python3.10/site-p

#### Due to resource constraint, i can only train it for 10 epochs, you can train more epochs for better result

In [28]:
test(model, test_images, test_labels)

Test Accuracy: 27.26%, Test F1-score: 0.25


# Flower Framework

In [17]:
# Define the Flower server with a simple federated averaging strategy
def start_server():
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=3,  # Require 3 clients to be available
        min_fit_clients=3,        # Minimum number of clients for training in each round
    )
    fl.server.start_server(server_address="127.0.0.1:8080", strategy=strategy)

# Start the server (execute this cell to start the server in Jupyter)


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
from skimage.feature import hog
import torch.utils.data as data

# Define the Softmax Regression model
class SoftmaxRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, X):
        return self.linear(X)

# Define the Flower client for federated learning
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_data, train_labels, val_data, val_labels):
        self.model = model
        self.train_data = train_data
        self.train_labels = train_labels
        self.val_data = val_data
        self.val_labels = val_labels
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)
    
    def get_parameters(self):
        return [val.cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        for param, new_param in zip(self.model.parameters(), parameters):
            param.data = torch.tensor(new_param, dtype=param.dtype)
    
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(1):  # One epoch of local training
            outputs = self.model(self.train_data)
            loss = self.criterion(outputs, self.train_labels)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        return self.get_parameters(), len(self.train_data), {}
    
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        outputs = self.model(self.val_data)
        loss = self.criterion(outputs, self.val_labels)
        accuracy = (outputs.argmax(1) == self.val_labels).sum().item() / len(self.val_labels)
        return loss.item(), len(self.val_data), {"accuracy": accuracy}

# Load your preprocessed HOG data for each client
def load_data(client_id):
    if client_id == 1:
        train_data, train_labels = client2_train_images, client2_train_labels  # Client 1's dataset
    elif client_id == 2:
        train_data, train_labels = client2_train_images, client2_train_labels      # Client 2's dataset
    elif client_id == 3:
        train_data, train_labels = client2_train_images, client2_train_labels    # Client 3's dataset
    val_data, val_labels = client2_train_images, client2_train_labels          # Validation data (same for all)
    return train_data, train_labels, val_data, val_labels

# Start the client (each client runs in a different Jupyter cell or process)
def start_client(client_id):
    train_data, train_labels, val_data, val_labels = load_data(client_id)
    
    # Define model
    model = SoftmaxRegression(input_dim=324, output_dim=10)
    
    # Convert data to tensors
    train_data = torch.tensor(train_data, dtype=torch.float32)
    train_labels = torch.tensor(train_labels, dtype=torch.long)
    val_data = torch.tensor(val_data, dtype=torch.float32)
    val_labels = torch.tensor(val_labels, dtype=torch.long)

    # Create Flower client
    client = FlowerClient(model, train_data, train_labels, val_data, val_labels)
    flwr_client = client.to_client()
    # Start the Flower client using the new method
    fl.client.start_client(server_address="127.0.0.1:8080", client=flwr_client)

# # Example to start a client (execute one cell per client)
# start_client(1)  # Client 1
# start_client(2)  # Client 2
# start_client(3)  # Client 3
# start_server()

In [21]:
# Creating threads for server and clients
def run_federated_training():

    # Start server thread
    server_thread = threading.Thread(target=start_server, args=())
    server_thread.start()
    time.sleep(1)  # Wait a moment to ensure server starts before clients connect

    # Start client threads
    client_threads = [
        threading.Thread(target=start_client, args=(1,)),
        threading.Thread(target=start_client, args=(2,)),
        threading.Thread(target=start_client, args=(3,)),
    ]
    for thread in client_threads:
        thread.start()
    # Join threads to ensure complete execution
    for thread in client_threads:
        thread.join()
    server_thread.join()

In [22]:
run_federated_training()

INFO :      Starting Flower server, config: num_rounds=1, no round_timeout
/tmp/ipykernel_506/2626029719.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor(train_data, dtype=torch.float32)
/tmp/ipykernel_506/2626029719.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.long)
/tmp/ipykernel_506/2626029719.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_data = torch.tensor(val_data, dtype=torch.float32)
/tmp/ipykernel_506/2626029719.py:76: UserWarni